In [91]:
# Integrate and pick out suitable PTT contents for training RNN and seq2seq model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
totalboard_name = 'BG-CC-MV-SX-TJ-WT'
board_name = ['Boy-Girl','C_Chat','movie','sex','Tech_job','WomenTalk']
board_dict = {'Boy-Girl':'1520323590.6503718','C_Chat':'1520385092.0256078-merge',
              'movie':'1520492468.9048126-merge','sex':'1520269282.2598605-merge',
              'Tech_job':'1519748408.0598423','WomenTalk':'1520349866.2532854'}
board_dict_index = {'Tech_job':0,'Boy-Girl':1,'AllTogether':2,'sex':3,'C_Chat':4,'movie':5}
remove_type = ['無題', '轉錄', '轉載', '轉發', '轉貼', '轉潑', '轉PO', '公告', '判決', '申請', '申訴', '水桶', '教學', 'ANSI', '市調', '問卷', '廣告', '協尋', '尋人']
max_word_length = 250
min_word_length = 100
max_title_length = 20
min_title_length = 4
id_len = 6
label_newnum = len(board_name)

board_folder = []
board_dir = []
content_dir = []
push_dir = []
new_content_dir = []
new_push_dir = []
totalboard_dir = os.path.join(base_dir, totalboard_name)
if not os.path.exists(totalboard_dir):
    os.makedirs(totalboard_dir)

for name_ind in range(len(board_name)):
    # Original data path.
    board_folder.append(board_name[name_ind] + '_' + board_dict[board_name[name_ind]])
    board_dir.append(os.path.join(base_dir, board_folder[name_ind]))
    if not os.path.exists(board_dir[name_ind]):
        os.makedirs(board_dir[name_ind])
    content_dir.append(os.path.join(board_dir[name_ind], 'content'))
    if not os.path.exists(content_dir[name_ind]):
        os.makedirs(content_dir[name_ind])
    push_dir.append(os.path.join(board_dir[name_ind], 'push'))
    if not os.path.exists(push_dir[name_ind]):
        os.makedirs(push_dir[name_ind])
    # New data path
    new_content_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_content'))
    if not os.path.exists(new_content_dir[name_ind]):
        os.makedirs(new_content_dir[name_ind])
    new_push_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_push'))
    if not os.path.exists(new_push_dir[name_ind]):
        os.makedirs(new_push_dir[name_ind])

In [92]:
#整理、修改各大看版原始數據
import pandas as pd
import numpy as np
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
import jieba

# Import dictionary fron jieba and PTT.
jieba.set_dictionary('C:/Users/User/Anaconda3/Lib/site-packages/jieba/dict.txt.big')
jieba.load_userdict('C:/Users/User/Anaconda3/Lib/site-packages/jieba/userdict.txt')

board_post_num = []
board_word_num_tpye = ['min','q10','q25','q50','q75','q90','max','mean','std']
board_word_num = {'min':[],'q10':[],'q25':[],'q50':[],'q75':[],'q90':[],'max':[],'mean':[],'std':[]}

charabc = 'abcdefghijklmnopqrstuvwxyz'
char123 = '0123456789'
charabc123_dict = {}
for char in charabc:
    charabc123_dict[ord(char)] = None
for char in char123:
    charabc123_dict[ord(char)] = None

# 如果輸入字串推測像ID (長度大於等於6且由字母與數字組合)，則回傳True
def is_personal_id(content_text):
    return len(content_text)>=id_len and content_text.strip(charabc)!='' and content_text.strip(char123)!='' and content_text.translate(charabc123_dict)==''

#filters_char='`|＼｜＊／＝≦≧＿＠＃＄％︿＆～§◎．※↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔'
filters_char='ˊˋ#*×/\\^_`|＼﹨｜〝〞‵′＊／≦≧＿＃⌒‿．§◎˙※ㄧ↔│ꄊ✂☁☛✈♥○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓░▒╱╲╳▁━▶▁▂▃▄▅▆▇█▉▊▋▋▌▍▎▏▐▔■▁▔' 
filters_dict = {ord('Ａ'):'a', ord('ａ'):'a', ord('Ｂ'):'b', ord('ｂ'):'b', 
                ord('Ｃ'):'c', ord('ｃ'):'c', ord('Ｄ'):'d', ord('ｄ'):'d', 
                ord('Ｅ'):'e', ord('ｅ'):'e', ord('Ｆ'):'f', ord('ｆ'):'f', 
                ord('Ｇ'):'g', ord('ｇ'):'g', ord('Ｈ'):'h', ord('ｈ'):'h', 
                ord('Ｉ'):'i', ord('ｉ'):'i', ord('Ｊ'):'j', ord('ｊ'):'j', 
                ord('Ｋ'):'k', ord('ｋ'):'k', ord('Ｌ'):'l', ord('ｌ'):'l', 
                ord('Ｍ'):'m', ord('ｍ'):'m', ord('Ｎ'):'n', ord('ｎ'):'n', 
                ord('Ｏ'):'o', ord('ｏ'):'o', ord('Ｐ'):'p', ord('ｐ'):'p', 
                ord('Ｑ'):'q', ord('ｑ'):'q', ord('Ｒ'):'r', ord('ｒ'):'r', 
                ord('Ｓ'):'s', ord('ｓ'):'s', ord('Ｔ'):'t', ord('ｔ'):'t', 
                ord('Ｕ'):'u', ord('ｕ'):'u', ord('Ｖ'):'v', ord('ｖ'):'v', 
                ord('Ｗ'):'w', ord('ｗ'):'w', ord('Ｘ'):'x', ord('ｘ'):'x', 
                ord('Ｙ'):'y', ord('ｙ'):'y', ord('Ｚ'):'z', ord('ｚ'):'z',
                ord('０'):'0', ord('１'):'1', ord('２'):'2', ord('３'):'3', 
                ord('４'):'4', ord('５'):'5', ord('６'):'6', ord('７'):'7', 
                ord('８'):'8', ord('９'):'9', ord('＋'):'+', ord('－'):'-', 
                ord('％'):'%', ord('＆'):'&', ord('＄'):'$', ord('＝'):'=', 
                ord('＠'):'@'}
for char in filters_char:
    filters_dict[ord(char)] = None

split_char = ',.!?$'
dot_char = '。\n~～'
comma_char = '，、 　\t；;：:"'
upper_brackets_char = '<([{＜﹝「『（｛［【《〖﹙“‘︵︷︹︻︽︿﹁﹃'
lower_brackets_char = '>)]}＞﹞」』）｝］】》〗﹚”’︶︸︺︼︾﹀﹂﹄'
split_dict = {ord('！'):'!', ord('？'):'?'}
for char in dot_char:
    split_dict[ord(char)] = '.'
for char in comma_char:
    split_dict[ord(char)] = ','
for char in upper_brackets_char:
    split_dict[ord(char)] = ','
for char in lower_brackets_char:
    split_dict[ord(char)] = ','

def del_char(chars, del_index):
    if del_index+1>=len(chars):
        chars = chars[:del_index]
    elif del_index==0:
        chars = chars[1:]
    else:
        chars = chars[:del_index]+chars[del_index+1:]
    return chars

def del_same_split(chars):
    is_same = 0
    for i in range(len(chars)-1,0,-1):
        if chars[i] == chars[i-1]:
            is_same+=1
            if is_same>2:
                chars = del_char(chars, i)
            else:
                for char in split_char:
                    if chars[i]==char:
                        chars = del_char(chars, i)
                        break
        else:
            is_same=0
    return chars

# 刪除輸入字串中的網址和特殊符號，將全形數字字母轉換成半形
def arrange_content(content_text):
    # Remove http line in content.
    http_local = content_text.rfind('http')
    while http_local >= 0:
        http_str = content_text.rfind('\n',0 , http_local)
        if http_str < 0:
            http_str = 0
        http_end = content_text.find('\n', http_local)
        if http_end < 0:
            http_end = len(content_text)
        content_text = content_text.replace(content_text[http_str:http_end], '')
        http_local = content_text.rfind('http', 0, http_str)
        
    # Remove filtered CHARs in content.
    content_text = content_text.translate(filters_dict)
    content_text = content_text.translate(split_dict)
    content_text = del_same_split(content_text)
    
    # Word length filter by jieba words slit.
    text_jieba = jieba.cut(content_text, cut_all=False)
    content_text = ''
    word_length = 0
    for word in text_jieba:
        if is_personal_id(word):
            word = 'nameid' 
        word_length += 1
        content_text = content_text + ' ' + word
    return content_text

def arrange_push(push_input):
    # Read push content and remove empty.
    pushs = push_input.split('\n')
    push_type = []
    push_id = []
    push_content = []
    push_time = []
    for push in pushs:
        push = push.split(',')
        if len(push)>4:
            try:
                strptime = time.strptime(push[4],' %m/%d %H:%M')
            except BaseException:
                continue
            if (len(push_id)==0 or push_id[-1]!=push[2]):
                push_time.append(strptime)
                if push[1].rfind('推')>=0:
                    push_type.append(1)
                elif push[1].rfind('噓')>=0:
                    push_type.append(-1)
                else:
                    push_type.append(0)
                push_id.append(push[2])
                push_content.append(arrange_content(push[3].lower()))

            else:
                try:
                    push_time[-1] = strptime
                except BaseException:
                    continue
                if push[1].rfind('推')>=0:
                    push_type[-1] += 1
                elif push[1].rfind('噓')>=0:
                    push_type[-1 ]+= -1
                push_content[-1]+arrange_content(push[3].lower())
    return push_type, push_id, push_content, push_time


for name_ind in range(len(board_name)):
    dfs = pd.read_csv(os.path.join(board_dir[name_ind], board_dict[board_name[name_ind]] + '.csv'))
    board_post_num.append(len(dfs))
    print('Running ..., board:', board_name[name_ind])
    print('Original post number:', board_post_num[name_ind])
    for dfs_index in range(board_post_num[name_ind]-1, -1, -1):
        read_index = int(dfs.iloc[dfs_index, 0])
        type_name = str(dfs.iloc[dfs_index, 1])
        word_length = int(dfs.iloc[dfs_index, 5])
        push_length = int(dfs.iloc[dfs_index, 6])
        title_name = str(dfs.iloc[dfs_index, 9])
        title_length = len(title_name)
        
        # Remove data if there was no push.
        if push_length<=1:
            dfs.drop([dfs_index], inplace=True)
            continue
            
        # Post type filter
        is_continue = False
        for remove in remove_type:
            if type_name.find(remove)>=0:
                is_continue = True
                break
                
        # Remove data if the type was not required.
        if is_continue:
            dfs.drop([dfs_index], inplace=True)
            continue
            
        # Remove data if title length was false.
        '''if title_length > max_title_length or title_length < min_title_length:
            dfs.drop([dfs_index], inplace=True)
            continue'''
        
        # Remove data if content word length was false.
        if word_length > max_word_length*4 or word_length < min_word_length:
            dfs.drop([dfs_index], inplace=True)
            continue
        
        # Read csv and remove file which was not existed.
        text = ''
        with open(os.path.join(content_dir[name_ind], str(read_index) + '.csv'), 
                    'r', encoding = 'utf-8-sig') as file:
            csvCursor = csv.reader(file)
            for rows in csvCursor:
                for row in rows:
                    # Read content and remove empty.
                    row = arrange_content(row.lower())
                    text = text + row
        
        # Remove data if content word length was false after jieba slitting.
        if word_length > max_word_length or word_length < min_word_length:
            dfs.drop([dfs_index], inplace=True)
            continue
        
        # Read csv and arrange push content.
        with open(os.path.join(push_dir[name_ind], str(read_index) + '.csv'),
                    'r', encoding = 'utf-8-sig') as file:
            csvCursor = csv.reader(file)
            for rows in csvCursor:
                for row in rows:
                    push_type, push_id, push_content, push_time = arrange_push(row)
            push_length = len(push_content)
                    
        # Save posts after arrange post content.
        with open(os.path.join(new_content_dir[name_ind], str(read_index) + '.csv'), 
                  'w', encoding = 'utf-8-sig') as file:
            csvCursor = csv.writer(file)
            csvCursor.writerow([text])
        dfs.iloc[dfs_index, 5] = word_length
            
        # Save posts after arrange push content.
        with open(os.path.join(new_push_dir[name_ind], str(read_index) + '.csv'), 
                  'w', encoding = 'utf-8-sig') as file:
            csvCursor = csv.writer(file)
            csvCursor.writerow(push_content)
        dfs.iloc[dfs_index, 6] = push_length
        
        if not is_continue and dfs_index % 100 == 0:
            print("index: {:0>6d}".format(dfs_index), end='\r')
    
    dfs.to_csv(os.path.join(new_content_dir[name_ind], board_name[name_ind] + '_fix.csv'), 
               encoding = 'utf-8-sig', index=False)
    
    board_post_num[name_ind] = (len(dfs))
    board_word_num['min'].append(dfs.iloc[:, 5].min())
    board_word_num['q10'].append(dfs.iloc[:, 5].quantile(0.1))
    board_word_num['q25'].append(dfs.iloc[:, 5].quantile(0.25))
    board_word_num['q50'].append(dfs.iloc[:, 5].quantile(0.5))
    board_word_num['q75'].append(dfs.iloc[:, 5].quantile(0.75))
    board_word_num['q90'].append(dfs.iloc[:, 5].quantile(0.9))
    board_word_num['max'].append(dfs.iloc[:, 5].max())
    board_word_num['mean'].append(dfs.iloc[:, 5].mean())
    board_word_num['std'].append(dfs.iloc[:, 5].std())
    
    print("This board was end execution, post number:", board_post_num[name_ind])

print('Board name:', board_name)
print('Post number:', board_post_num)
for qtype in board_word_num_tpye:
    print(qtype, ': ', board_word_num[qtype])

Building prefix dict from C:\Users\User\Anaconda3\Lib\site-packages\jieba\dict.txt.big ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.ufa6ae29b0cbce8b45e006c7fa30eaaf8.cache
Loading model cost 0.967 seconds.
Prefix dict has been built succesfully.


Running ..., board: Boy-Girl
Original post number: 75291
This board was end execution, post number: 9172
Running ..., board: C_Chat
Original post number: 280564
This board was end execution, post number: 68066
Running ..., board: movie
Original post number: 128246
This board was end execution, post number: 26451
Running ..., board: sex
Original post number: 71154
This board was end execution, post number: 11082
Running ..., board: Tech_job
Original post number: 58934
This board was end execution, post number: 15327
Running ..., board: WomenTalk
Original post number: 128693
This board was end execution, post number: 44379
Board name: ['Boy-Girl', 'C_Chat', 'movie', 'sex', 'Tech_job', 'WomenTalk']
Post number: [9172, 68066, 26451, 11082, 15327, 44379]
min :  [100, 100, 100, 100, 100, 100]
q10 :  [119.0, 113.0, 112.0, 113.0, 109.0, 114.0]
q25 :  [145.0, 133.0, 131.0, 133.0, 125.0, 133.0]
q50 :  [181.0, 167.0, 166.0, 168.0, 156.0, 165.0]
q75 :  [216.0, 206.0, 204.0, 206.0, 197.0, 203.0]
q9

In [93]:
#整合各大看板資料，挑選訓練、測試用數據
import pandas as pd
import numpy as np
import random
board_number_limit = [9088,21632,18688,11008,15232,20352] #[9088,67968,26368,11008,15232,44288]
board_post_num = []

# Select content each board by number=board_number_limit.
for name_ind in range(len(board_name)):
    dfs = pd.read_csv(os.path.join(new_content_dir[name_ind], board_name[name_ind] + '_fix.csv'))
    board_post_num.append(len(dfs))
    random_board = list(range(board_post_num[name_ind]))
    random.shuffle(random_board)
    for dfs_index in range(board_post_num[name_ind]):
        if random_board[dfs_index] >= board_number_limit[name_ind]:
            dfs.drop([dfs_index], inplace=True)
    dfs.to_csv(os.path.join(new_content_dir[name_ind], board_name[name_ind] + '_select.csv'), 
               encoding = 'utf-8-sig', index=False)

# Merge data each board
for name_ind in range(len(board_name)):
    dfs = pd.read_csv(os.path.join(new_content_dir[name_ind], board_name[name_ind] + '_select.csv'))
    dfs.loc[:,'board_name'] = pd.Series(np.full(board_post_num[name_ind], board_name[name_ind]))
    if name_ind > 0:
        dfs_total = pd.concat([dfs_total, dfs])
    else:
        dfs_total = dfs

# Upset and save merged data
dfs_total.reset_index(drop=True ,inplace=True)
dfs_total = dfs_total.sample(frac= 1.0 ,replace=True)
dfs_total.to_csv(os.path.join(totalboard_dir, totalboard_name + '.csv'), 
                 encoding = 'utf-8-sig', index=False)